# L1: Information retrieval

In this lab you will apply basic techniques from information retrieval to implement the core of a minimalistic search engine. The data for this lab consists of a collection of app descriptions scraped from the [Google Play Store](https://play.google.com/store/apps?hl=en). From this collection, your search engine should retrieve those apps whose descriptions best match a given query under the vector space model.

## Data set

The app descriptions come in the form of a compressed [JSON](https://en.wikipedia.org/wiki/JSON) file. Start by loading this file into a [Pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe).

In [1]:
import bz2
import pandas as pd

with bz2.open('app-descriptions.json.bz2') as source:
    df = pd.read_json(source)

In Pandas, a DataFrame is a table with indexed rows and labelled columns of potentially different types. Data in a DataFrame can be accessed in various ways, including by row and by column. To give an example, the code in the next cell shows rows 200–204:

In [2]:
df[200:205]

,name,description
200,Brick Breaker Star: Space King,Introducing the best Brick Breaker game that e...
201,Brick Classic - Brick Game,Classic Brick Game!\n\nBrick Classic is a popu...
202,Bricks Breaker - Glow Balls,Bricks Breaker - Glow Balls is a addictive and...
203,Bricks Breaker Quest,How to play\n- The ball flies to wherever you ...
204,Brothers in Arms® 3,Fight brave soldiers from around the globe on ...


As you can see, there are two labelled columns: `name` (the name of the app) and `description` (a textual description). The code in the next cell shows how to acess fields from the description column.

In [3]:
df['description'][200:205]

200    Introducing the best Brick Breaker game that e...
201    Classic Brick Game!\n\nBrick Classic is a popu...
202    Bricks Breaker - Glow Balls is a addictive and...
203    How to play\n- The ball flies to wherever you ...
204    Fight brave soldiers from around the globe on ...
Name: description, dtype: object

## Problem 1: Preprocessing

Your first task is to implement a preprocessor for your search engine. In the vector space model, *preprocessing* refers to any kind of transformation that is applied before a text is vectorized. Here you can restrict yourself to a very simple preprocessing: tokenization, stop word removal, and lemmatization.

To implement your preprocessor, you can use [spaCy](https://spacy.io). Make sure that you read the [Linguistic annotations](https://spacy.io/usage/spacy-101#annotations) section of the spaCy&nbsp;101; that section contains all the information that you need for this problem (and more).

Implement your preprocessor by completing the skeleton code in the next cell, adding additional code as you feel necessary.

In [4]:
import spacy

def preprocess(text):
   
    # Load Model
    nlp = spacy.load('en_core_web_sm')
    
    # tokenization, stop word removal and lemmatization and discarding non-alphabetical characters  
    return [token.lemma_ for token in nlp(text) if token.lemma_.isalpha() and not token.is_stop]

Your implementation should conform to the following specification:

<strong>preprocess</strong> (<em>text</em>)

> Preprocesses given text by tokenizing it, removing any stop words, replacing each remaining token with its lemma (base form), and discarding all lemmas that contain non-alphabetical characters. Returns the list of remaining lemmas (represented as strings).

**Tip:** To speed up the preprocessing, you can disable loading those spaCy components that you do not need, such as the part-of-speech tagger, parser, and named entity recognizer. See [here](https://spacy.io/usage/processing-pipelines#disabling) for more information about this.

Test your implementation by running the following cell:

In [5]:
preprocess('Apple is looking at buying U.K. startup for $1 billion')

['Apple', 'look', 'buy', 'startup', 'billion']

This should give the following output:


## Problem 2: Vectorizing

Your next task is to vectorize the data – and more specifically, to map each app description to a tf–idf vector. For this you can use the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) class from [scikit-learn](https://scikit-learn.org/stable/). Make sure to specify your preprocessor from the previous problem as the `tokenizer` &ndash; not the `preprocessor`! &ndash; for the vectorizer. (In scikit-learn parlance, the `preprocessor` handles string-level preprocessing.)

In [6]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
vectorizer = TfidfVectorizer(tokenizer=preprocess)
X = vectorizer.fit_transform(df['description'])

Test your implementation by running the following cell:

In [8]:
X.shape

(1614, 21676)

This should show the dimensions of the matrix `X` to be 1614 × 20669.

## Problem 3: Retrieving

To complete the search engine, your last task is to write a function that returns the most relevant app descriptions for a given query. An easy way to do solve this task is to use scikit-learn&rsquo;s [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) class. That class implements unsupervised nearest neighbours learning, and allows you to easily find a predefined number of app descriptions whose vector representations are closest to the query vector.

In [9]:
from sklearn.neighbors import NearestNeighbors

# Train model
knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(X)

def search(query):
    # TODO: Replace the next line with your own code.
    input_features = vectorizer.transform([query])
    N = knn.kneighbors(input_features, return_distance=False)
    
    return df.iloc[N[0]]


Your implementation should conform to the following specification:

<strong>search</strong> (<em>query</em>)

> Returns the 10 app descriptions most similar (in terms of cosine similarity) to the given query as a Pandas DataFrame.

Test your implementation by running the following cell:

In [10]:
search('dodge trains')

,name,description
998,No Humanity - The Hardest Game,2M+ Downloads All Over The World!\n\n* IGN Nom...
1300,Subway Princess Runner,"Subway princess runner, Bus run, forest rush w..."
1168,Rush,Are you ready for a thrilling ride?\n\nRush th...
1301,Subway Surfers,DASH as fast as you can! \nDODGE the oncoming ...
1465,Virus War - Space Shooting Game,Warning! Virus invasion! Destroy them with you...
1153,Road Riot,Road Riot is the global sensation that defined...
360,Dancing Road: Color Ball Run!,Try out the most exciting Running - Sliding - ...
757,Kids Numbers and Math,Wouldn't it be just wonderful if there was a s...
384,Dialog Mega Run,Run around the iconic places of Sri Lanka !\nD...
584,Galaxiga - Classic 80s Arcade,Galaxiga is a best space shooting game that ma...


The top hit in the list should be *Subway Surfers*.

## Problem 4: Finding terms with low/high idf

Recall that the inverse document frequency (idf) of a term is the lower the more documents from a given collection the term appears in. To get a better understanding for this concept, your next task is to write code to find out which terms have the lowest/highest idf with respect to the app descriptions.

Start by sorting the terms in increasing order of idf, breaking ties by falling back on alphabetic order.

In [11]:
# TODO: Replace the next line with your own code.

feature_names = vectorizer.get_feature_names() # All the words
feature_idf = vectorizer.idf_ # idf values of the All the words

zipped = zip(feature_names, feature_idf) # map each word to its idf value
terms = dict(sorted(zipped, key = lambda t: t[1])) # sort in ascending idf value - this is a list

Now, print the 10 terms with the lowest/highest idf. How do you explain the results?

In [12]:
# Highest idf
print(list(terms.items())[-10:])

[('회원가입에', 7.6939430550968115), ('회원을', 7.6939430550968115), ('획득한', 7.6939430550968115), ('효과', 7.6939430550968115), ('효과음', 7.6939430550968115), ('ﬁnd', 7.6939430550968115), ('ﬁnger', 7.6939430550968115), ('ﬁnish', 7.6939430550968115), ('ﬁrst', 7.6939430550968115), ('ﬂye', 7.6939430550968115)]


In [13]:
# Lowest idf
print(list(terms.items())[:10])

[('game', 1.2970133998806652), ('play', 1.4201230970427738), ('feature', 1.494448594484668), ('free', 1.5846954723324462), ('new', 1.6656645348661132), ('world', 1.781792314708555), ('time', 1.8106206666085325), ('app', 1.8388711328943845), ('fun', 1.8591323180342063), ('use', 1.87386012474445)]


**Answer** : TF-IDF assigns more weight to less frequently occurring words rather than frequently occurring ones. It is based on the assumption that less frequently occurring words are more important. In particular, words like: game, fun or app, are very frequent in descriptions of games in the Google Play Store. However, words like zac, or zazz are less frequent, therefore it is assumed that this words have a bigger impact on the meaning of the sentence.

## Problem 5: Keyword extraction

A simple method for extracting salient keywords from a document is to pick the $k$ terms with the highest tf–idf value. Your last task in this lab is to implement this method. More specifically, we ask you to implement a function `keywords` that extracts keywords from a given text.

In [14]:
def keywords(text, n=10):
    
    # Create dictionary with words in description and its idf value
    w_idf = {word:terms[word] for word in preprocess(text) if word in terms}
    
    # Sort dictionary by value and keep n first wordsd
    sorted_w_idf = sorted(w_idf.items(), key=lambda x: x[1], reverse=True)[:n]
    
    return [word[0] for word in sorted_w_idf]

Your implementation should conform to the following specification:

<strong>keywords</strong> (<em>text</em>, <em>n</em> = 10)

> Returns a list with the $n$ (default value: 10) most salient keywords from the specified text, as measured by their tf–idf value relative to the collection of app descriptions.

Test your implementation by running the following cell:

In [15]:
print(keywords(df['description'][1428]))

['timetable', 'haul', 'railyard', 'overcast', 'tram', 'locomotive', 'breakneck', 'thunderstorm', 'diesel', 'fork']


This should give the following output:

<div class="alert alert-info">
    Please read the General information section on the lab web page before submitting this notebook!
</div>